## Input Parameters

In [1]:
# The default is to include points from all NLCD classes and MGRS grid zones
point_nlcd_keep_list = []
point_mgrs_keep_list = []

# # This would show all NLCD classes for MGRS grid zone 12T
# point_nlcd_keep_list = [31, 41, 42, 43, 52, 71, 81, 82, 90, 95]
# point_mgrs_keep_list = ['12T']

# This would show NLCD 82 points for all MGRS grid zones
point_nlcd_keep_list = [82]
point_mgrs_keep_list = []

# The minimum number of months in the target year
min_month_count = 3

# Overwrite the summary stats file everytime this block is run
output_txt = 'summary_stats_nlcd82.txt'
output_f = open(output_txt, 'w')
output_f.write('MGRS: '+ ', '.join(point_mgrs_keep_list) + '\n')
output_f.write('NLCD: '+ ', '.join(map(str, point_nlcd_keep_list)) + '\n\n')
output_f.close()

## Other Parameters

In [2]:
data_folder = 'data'
points_folder = 'points'

points_csv = 'gap_fill_test_points.csv'

# Exclude 2016 from the statistics since there is not a full prior year to interpolate from
# Including 2024 even though 2025 is not complete
stats_years = list(range(2017, 2025))

# Points were only built for NLCD 2024
nlcd_years = [2024]

# Include all MGRS grid zones except 12R and 16U since they are too small
mgrs_zones = [
    '10S', '10T', '10U', '11S', '11T', '11U', '12S', '12T', '12U', 
    '13R', '13S', '13T', '13U', '14R', '14S', '14T', '14U', '15R', '15S', '15T', '15U', 
    '16R', '16S', '16T', '17R', '17S', '17T', '18S', '18T', '19T'
    # '12R', '16U'
]

months = list(range(1, 13))

# TODO: Add support for setting a minimum number of months in the year
#   and minimum number of months in the growing season
# min_month_count = 6
# min_gs_month_count = 3


## Python Imports

In [3]:
from datetime import datetime
import os
import pprint
import random

#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import seaborn as sns
import sklearn.metrics
import sklearn.linear_model

from whittaker_eilers import WhittakerSmoother

## Read the point CSV files

In [4]:
# Building a single points dataframe and CSV from the MGRS grid zone points CSV files
overwrite_flag = False

# Read the separate points CSV files into a single dataframe
points_df_list = [
    pd.read_csv(os.path.join(points_folder, f'points_{mgrs_zone}_{nlcd_year}.csv'), index_col=None, header=0)
    for nlcd_year in nlcd_years
    for mgrs_zone in mgrs_zones
    if os.path.isfile(os.path.join(points_folder, f'points_{mgrs_zone}_{nlcd_year}.csv'))
]
points_df = pd.concat(points_df_list, axis=0, ignore_index=True)
print(f'Points: {len(points_df.index)}')

# The mgrs_zone value will eventually be added to the csv files
points_df['mgrs_zone'] = points_df['mgrs_tile'].str.slice(0, 3)

# Add a unique index to the points dataframe
points_df['index_group'] = points_df.groupby(['mgrs_tile', 'nlcd']).cumcount()
points_df['point_id'] = (
    points_df["mgrs_tile"].str.upper() + '_' +
    'nlcd' + points_df["nlcd"].astype(str).str.zfill(2) + '_' +
    points_df["index_group"].astype(str).str.zfill(2)
)
del points_df['index_group']

# Round the lat and lon to 8 decimal places (probably should be 6)
points_df['latitude'] = round(points_df['latitude'], 8)
points_df['longitude'] = round(points_df['longitude'], 8)

# # Write to CSV
# if not os.path.isfile(points_csv) or overwrite_flag:
#     print('Writing points csv')
#     points_df.to_csv(points_csv, index=False)


Points: 17084


## Read the data CSV files

This block may take a little while to execute

In [5]:
# Read the CSV files into separate dataframes for each point
print('Reading mgrs data csv files')
data_df_dict = {}
for mgrs_zone in mgrs_zones:
    # print(mgrs_zone)
    if not os.path.isfile(os.path.join(data_folder, f'data_{mgrs_zone}.csv')):
        continue
        
    mgrs_df = pd.read_csv(os.path.join(data_folder, f'data_{mgrs_zone}.csv'), index_col=None, header=0)

    # Set MGRS value to upper case 
    # (at some point change this in all the data CSV files)
    mgrs_df['mgrs_tile'] = mgrs_df['mgrs_tile'].str.upper()
    mgrs_df['mgrs_zone'] = mgrs_df['mgrs_zone'].str.upper()
    
    # Compute the ET fraction
    mgrs_df['etf'] = mgrs_df['et'] / mgrs_df['eto']
    
    # Get the month for computing climos
    mgrs_df['date'] = pd.to_datetime(mgrs_df['date'])
    mgrs_df['year'] = mgrs_df['date'].dt.year
    mgrs_df['month'] = mgrs_df['date'].dt.month
    
    # Confirm that specific NLCD categories are not included
    # TODO: This probably isn't needed and switch to a check instead of masking
    for nlcd_skip in [11, 12, 21, 22, 23]:
        mgrs_df = mgrs_df[mgrs_df['nlcd'] != nlcd_skip]

    # Save dataframe for each point
    for point_id in mgrs_df['point_id'].unique():
        site_df = mgrs_df.loc[mgrs_df['point_id']==point_id].copy()
        site_df.set_index('date', drop=True, inplace=True)
        site_df.sort_index(inplace=True)
        data_df_dict[point_id] = site_df
        del site_df
    del mgrs_df

print('\nDone')

Reading mgrs data csv files

Done


## Compute the ETf climos

In [6]:
# Compute the maximum ETf per site
# Assuming it is okay to make this for the full period of record
print('\nComputing maximum ETf')
etf_max_dict = {
    point_id: data_df_dict[point_id].agg(etf=('etf', 'max'))['etf'].to_dict()['etf']
    for point_id in data_df_dict.keys()
}

# Compute climos for each site
# Only keep the climo value if there are at least "n" years of data
print('\nComputing monthly climatologies')
month_climo_count_min = 2
month_climo_dict = {}
for point_id in data_df_dict.keys():
    month_climo = data_df_dict[point_id].groupby(['month']).agg(
        etf=('etf', 'mean'), 
        etf_median=('etf', 'median'), 
        count=('etf', 'count'), 
        et=('et', 'mean'), 
        eto=('eto', 'mean'),
    )
    month_climo_count_mask = month_climo['count'] < month_climo_count_min
    month_climo.loc[month_climo_count_mask, ['etf', 'etf_median', 'et']] = np.nan
    month_climo_dict[point_id] = month_climo
    del month_climo, month_climo_count_mask

# # Compute climos for each target year that have the target year values excluded
# # CGM - This might be worth testing more but doesn't seem worth the time to generate
# print('\nComputing monthly climatologies for target years')
# month_climo_dict = {}
# for point_id in data_df_dict.keys():
#     if point_id not in month_climo_dict.keys():
#         month_climo_dict[point_id] = {}
#     for year in stats_years:
#         month_climo_dict[point_id][year] = (
#             data_df_dict[point_id][data_df_dict[point_id].year != year]
#             .groupby(['month'])
#             .agg(
#                 etf=('etf', 'mean'), 
#                 etf_median=('etf', 'median'), 
#                 count=('etf', 'count'), 
#                 et=('et', 'mean'), 
#                 eto=('eto', 'mean'),
#             )
#         )

# TODO: Join the climo values to the data dictionaries
#   It might be faster to join the climo data here instead of looking up in the function

print('\nDone')


Computing maximum ETf

Computing monthly climatologies

Done


## Functions for computing filled values and summary statistics

In [7]:
def generate_windows(
        point_id_list, 
        months=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
        years=[2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024], 
        exclude_months_without_climo=True
):
    """Generate Window Dataframes"""
    for i, point_id in enumerate(point_id_list):
        # Assume the data df dictionary exists in the global scope
        site_df = data_df_dict[point_id]
        
        for year in years:
            # Pull a three year window for each target year so that there are images to interpolate and fill from
            window_df = site_df[(site_df.index.year >= (year-1)) & (site_df.index.year <= (year+1))].copy()

            # If excluding months without climos, set them to NaN here
            # TODO: Test out adding the climo values to the window_df here 
            #   instead of in the compute_filled_values() function below
            if exclude_months_without_climo:
                merge_df = pd.merge(window_df[['month']], month_climo_dict[point_id]['etf'], how="left", on="month")
                climo_nan_mask = merge_df['etf'].isna().values
                window_df.loc[climo_nan_mask, ['etf', 'et', 'count']] = np.nan

            year_mask = window_df.index.year==year
            year_month_mask = year_mask & window_df.index.month.isin(months)
            
            if window_df.loc[year_mask, 'etf'].count() < min_month_count:
                # Check if there are enough months in the target year
                # print(f'{point_id} - {i} - {year} - not enough unmasked months, skipping')
                continue
            elif window_df.loc[year_month_mask, 'etf'].isna().all():
                # Check that there are target months with data in the 
                # print(f'{point_id} - {i} - {year} - no unmasked months in year/months, skipping')
                continue
            elif (window_df.loc[(window_df.index.year==year-1), 'etf'].isna().all() or 
                  window_df.loc[(window_df.index.year==year+1), 'etf'].isna().all()):
                # Check if there is data in the prev/next year to interpolate from
                # print(f'{point_id} - {i} - {year} - no unmasked months in next/prev year, skipping')
                continue

            yield point_id, year, window_df, year_month_mask


comparison_cols = [
    'interpolate', 'climo_mean', 'climo_median',
    'interp_clim_a', 'interp_clim_b', 'interp_clim_c',
    'whit_a_0p50', 'whit_a_0p20', 'whit_a_0p10', 'whit_a_0p05', 'whit_a_0p01', 
]


def compute_filled_values(window_df, tgt_indices, point_id):
    """"""
    # Get a copy of the target value before clearing
    original_etf = window_df.loc[tgt_indices, 'etf'].values
        
    # Set the target row values to NaN
    window_df.loc[tgt_indices, ('etf', 'et', 'count')] = np.nan
    
    # Setup the Whittaker Smoothing for the full dataframe outside of the index loop
    # The smoothing function needs all nans filled with a value
    # The fill value is not important as long as the weight value is set to 0
    window_df['temp'] = window_df['etf'].copy()
    window_df.loc[np.isnan(window_df['temp']), 'temp'] = -1
    etf = window_df['temp'].values

    # TODO: Make sure weights are set to 0 for all temp==-1 rows
    #   This might be happening already with the .fillna(0) call but double check
    #   Right now the code is assuming count is NaN if etf is NaN
    
    # Default weights with 1 for data and 0 for missing values
    weight_a = window_df['count'].clip(lower=1, upper=1).fillna(0)
    if not any(weight_a):
        print(f'{point_id} - {i} - {year} - all weights 0, skipping')
        return []
    # CGM - I tested out building the smoother once and then updating lambda,
    #   but it didn't seem any faster
    whit_a_0p50 = WhittakerSmoother(lmbda=0.5, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p20 = WhittakerSmoother(lmbda=0.2, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p10 = WhittakerSmoother(lmbda=0.1, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p05 = WhittakerSmoother(lmbda=0.05, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p01 = WhittakerSmoother(lmbda=0.01, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)

    # CGM - I was testing out trying different weights but it didn't seem to change the values at all
    # # Compute weights based on the the scene count value
    # # Set count 0 images to a weight of 0
    # weight = window_df['count'].clip(lower=0, upper=1).fillna(0)

    # # Compute weights based on the the scene count value
    # # Set counts of 0 to a weight of 0.5 and all other to 1    
    # weight = window_df['count'].add(1).clip(upper=2).divide(2).fillna(0)

    # # Compute weights based on the scene count value
    # # Set count weights as: 0 -> 0, 1 -> 0.5, 2+ -> 1
    # weight = window_df['count'].fillna(0).clip(upper=2).divide(2)

    # Process each target index separately
    values = []
    for i, (tgt_index, tgt_i) in enumerate(zip(tgt_indices, window_df.index.get_indexer(tgt_indices))):

        interp_value = window_df['etf'].interpolate(method='linear').loc[tgt_index]

        # Climos for all years
        climo_mean = month_climo_dict[point_id].loc[tgt_index.month, 'etf']
        climo_count = month_climo_dict[point_id].loc[tgt_index.month, 'count']
        climo_median = month_climo_dict[point_id].loc[tgt_index.month, 'etf_median']
        # # Climos with the target year excluded (not sure if this matters)
        # climo_mean = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'etf']
        # climo_count = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'count']
        # climo_median = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'etf_median']

        # Compute various combinations of averaging the climo and interpolate values
        # Simple mean
        interp_clim_a = (climo_mean + interp_value) / 2
        # Simple mean with the median climo
        interp_clim_c = (climo_median + interp_value) / 2
        # Weight the climo based on the number of months in the climo?
        climo_months = 10
        interp_clim_b = (climo_mean * climo_count + interp_value * climo_months) / (climo_count + climo_months)
            
        values.append({
            'index': tgt_index,
            'point_id': point_id,
            'mgrs': point_id.split('_')[0],
            'nlcd': int(point_id.split('_')[1][4:6]),
            'original': original_etf[i],
            # Filled values
            'interpolate': interp_value,
            'climo_mean': climo_mean,
            'climo_median': climo_median,
            'interp_clim_a': interp_clim_a,
            'interp_clim_b': interp_clim_b,
            'interp_clim_c': interp_clim_c,
            'whit_a_0p50': min(max(whit_a_0p50[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p20': min(max(whit_a_0p20[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p10': min(max(whit_a_0p10[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p05': min(max(whit_a_0p05[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p01': min(max(whit_a_0p01[tgt_i], 0), etf_max_dict[point_id]),
        })

    return values


def comparison_stats(df, x_col='original', y_cols=[], title='', print_flag=True, write_flag=True):
    """"""
    output = [title]

    # TODO: Build the format strings based on the number of parameters instead of hardcoding
    output.append('  {:>16s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s}'.format(
        'method', 'rmse', 'mae', 'mbe', 'm', 'b', 'r2', 'n'
    ))
    for y_col in y_cols:
        # Remove any NaN rows before computing statistics
        stat_df = df[df[y_col].notna()]
        model = sklearn.linear_model.LinearRegression()
        model.fit(stat_df[[x_col]], stat_df[y_col])

        output.append('  {:>16s} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8d}'.format(
            y_col,
            sklearn.metrics.root_mean_squared_error(stat_df[x_col], stat_df[y_col]),
            sklearn.metrics.mean_absolute_error(stat_df[x_col], stat_df[y_col]),
            np.mean(stat_df[y_col] - stat_df[x_col]),
            # np.mean(stat_df[x_col] - stat_df[y_col]),
            # sklearn.metrics.r2_score(stat_df[x_col], stat_df[y_col]),
            model.coef_[0],
            model.intercept_, 
            model.score(stat_df[[x_col]], stat_df[y_col]),
            # This count doesn't seem to change even when there are NaN values in the dataframe
            stat_df[y_col].count(),
        ))

    if print_flag:
        print('\n'.join(output))
    if write_flag:
        with open(output_txt, 'a') as output_f:
            output_f.write('\n'.join(output+ ['\n']))


## Filter to the target MGRS and NLCD classes

In [8]:
# Filter the points list to the target NLCD classes and MGRS grid zones
point_id_list = list(data_df_dict.keys())
if point_nlcd_keep_list:
    point_id_list = [p for p in point_id_list if int(p.split('_')[1][4:6]) in point_nlcd_keep_list]
if point_mgrs_keep_list:
    point_id_list = [p for p in point_id_list if p.split('_')[0][0:3] in point_mgrs_keep_list]

print(f'Points: {len(point_id_list)}')

Points: 1577


## Randomly drop one datapoint in each year

In [9]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop one datapoint in each year',
    print_flag=True, write_flag=True
)

Randomly drop one datapoint in each year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1385   0.1068  -0.0005   0.7445   0.1570   0.7379    12616
        climo_mean   0.1512   0.1157  -0.0003   0.6903   0.1906   0.6875    12616
      climo_median   0.1563   0.1110   0.0013   0.7284   0.1687   0.6708    12616
     interp_clim_a   0.1271   0.0997  -0.0004   0.7174   0.1738   0.7849    12616
     interp_clim_b   0.1276   0.0999  -0.0001   0.7174   0.1741   0.7828    12616
     interp_clim_c   0.1277   0.0976   0.0004   0.7364   0.1629   0.7795    12616
       whit_a_0p50   0.1446   0.1112  -0.0012   0.7489   0.1536   0.7157    12616
       whit_a_0p20   0.1399   0.1070  -0.0013   0.7865   0.1303   0.7358    12616
       whit_a_0p10   0.1386   0.1058  -0.0013   0.8063   0.1181   0.7428    12616
       whit_a_0p05   0.1385   0.1055  -0.0013   0.8194   0.1100   0.7452    12616
       whit_a_0p01   0.1395   0.1060  -0.0014   0.8327   

## Randomly drop a single datapoint from the "growing" season (Apr-Sept)

In [10]:
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
        
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a single datapoint from the "growing" season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a single datapoint from the "growing" season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1357   0.1052  -0.0112   0.7424   0.1651   0.7708    12616
        climo_mean   0.1545   0.1165   0.0035   0.6948   0.2122   0.6996    12616
      climo_median   0.1602   0.1114   0.0088   0.7401   0.1865   0.6825    12616
     interp_clim_a   0.1268   0.0991  -0.0038   0.7186   0.1886   0.8073    12616
     interp_clim_b   0.1269   0.0991  -0.0038   0.7186   0.1886   0.8072    12616
     interp_clim_c   0.1271   0.0963  -0.0012   0.7412   0.1758   0.8012    12616
       whit_a_0p50   0.1439   0.1113  -0.0059   0.7358   0.1747   0.7395    12616
       whit_a_0p20   0.1363   0.1042  -0.0038   0.7793   0.1472   0.7665    12616
       whit_a_0p10   0.1331   0.1013  -0.0030   0.8028   0.1319   0.7778    12616
       whit_a_0p05   0.1317   0.0999  -0.0026   0.8184   0.1215   0.7833    12616
       whit_a_0p01   0.1312 

## Randomly drop a single datapoint from the "non-growing" season (Oct-Mar)

In [11]:
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a single datapoint from the "non-growing" season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a single datapoint from the "non-growing" season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1425   0.1081   0.0179   0.7227   0.1632   0.6273    12582
        climo_mean   0.1468   0.1143  -0.0001   0.5788   0.2207   0.5827    12582
      climo_median   0.1508   0.1096  -0.0032   0.6054   0.2036   0.5630    12582
     interp_clim_a   0.1276   0.0989   0.0089   0.6508   0.1920   0.6883    12582
     interp_clim_b   0.1287   0.0995   0.0095   0.6515   0.1922   0.6820    12582
     interp_clim_c   0.1283   0.0976   0.0073   0.6640   0.1834   0.6825    12582
       whit_a_0p50   0.1479   0.1120   0.0103   0.7312   0.1512   0.6048    12582
       whit_a_0p20   0.1463   0.1107   0.0056   0.7518   0.1356   0.6160    12582
       whit_a_0p10   0.1466   0.1107   0.0034   0.7627   0.1278   0.6180    12582
       whit_a_0p05   0.1475   0.1113   0.0021   0.7700   0.1227   0.6167    12582
       whit_a_0p01   0.14

## Randomly drop a datapoint from the winter (Dec-Feb)

In [12]:
months = [12, 1, 2]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))

output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a datapoint from the winter (Dec-Feb)',
    print_flag=True, write_flag=True
)

Randomly drop a datapoint from the winter (Dec-Feb)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1433   0.1080   0.0194   0.6794   0.1727   0.5995    11022
        climo_mean   0.1467   0.1141   0.0041   0.5642   0.2124   0.5615    11022
      climo_median   0.1510   0.1091  -0.0009   0.5843   0.1978   0.5389    11022
     interp_clim_a   0.1287   0.0991   0.0117   0.6218   0.1925   0.6681    11022
     interp_clim_b   0.1309   0.1002   0.0132   0.6229   0.1935   0.6556    11022
     interp_clim_c   0.1297   0.0978   0.0093   0.6319   0.1852   0.6599    11022
       whit_a_0p50   0.1444   0.1088   0.0060   0.6873   0.1555   0.5909    11022
       whit_a_0p20   0.1452   0.1090   0.0039   0.7082   0.1433   0.5926    11022
       whit_a_0p10   0.1470   0.1101   0.0035   0.7188   0.1379   0.5883    11022
       whit_a_0p05   0.1491   0.1116   0.0034   0.7258   0.1345   0.5823    11022
       whit_a_0p01   0.1523   0.1140   0.0035 

## Randomly drop a single datapoint that is next to an existing missing data point 

In [13]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # Skip the year if there are no NaN values
    if not window_df.loc[year_month_mask, 'etf'].isna().any():
        # print(f'{point_id} - {i} - {year} - no missing data points, skipping')
        continue
    
    # For the target year, pick a random month that is next to a missing/masked month but has data
    nan_mask = window_df['etf'].isna()
    tgt_mask = (
        (nan_mask | nan_mask.shift(1) | nan_mask.shift(-1))
        & window_df['etf'].notna() & year_month_mask
    )
    if not tgt_mask.any():
        print(f'{point_id} - {i} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a single datapoint that is next to an existing missing data point',
    print_flag=True, write_flag=True
)

Randomly drop a single datapoint that is next to an existing missing data point
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1945   0.1518   0.0265   0.5478   0.2843   0.4027     5431
        climo_mean   0.1646   0.1273  -0.0101   0.5266   0.2598   0.5279     5431
      climo_median   0.1710   0.1230  -0.0137   0.5404   0.2483   0.4956     5431
     interp_clim_a   0.1608   0.1254   0.0082   0.5372   0.2720   0.5492     5431
     interp_clim_b   0.1642   0.1278   0.0102   0.5334   0.2763   0.5304     5431
     interp_clim_c   0.1623   0.1247   0.0064   0.5441   0.2663   0.5402     5431
       whit_a_0p50   0.2058   0.1602   0.0026   0.5886   0.2372   0.3775     5431
       whit_a_0p20   0.2050   0.1590  -0.0043   0.6202   0.2123   0.3943     5431
       whit_a_0p10   0.2059   0.1595  -0.0069   0.6366   0.2003   0.3998     5431
       whit_a_0p05   0.2074   0.1606  -0.0083   0.6475   0.1927   0.4007     5431
       whit_a_0p01

## Randomly drop a two month gap during the year

But only check the filled value in one month of the gap

In [14]:
dropped_months = 2
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))

    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a two month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a two month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1724   0.1327   0.0017   0.6472   0.2170   0.6023    12616
        climo_mean   0.1530   0.1171   0.0011   0.6806   0.1960   0.6839    12616
      climo_median   0.1582   0.1125   0.0021   0.7172   0.1747   0.6661    12616
     interp_clim_a   0.1423   0.1112   0.0014   0.6639   0.2065   0.7332    12616
     interp_clim_b   0.1432   0.1117   0.0019   0.6630   0.2075   0.7292    12616
     interp_clim_c   0.1425   0.1091   0.0019   0.6822   0.1958   0.7291    12616
       whit_a_0p50   0.1801   0.1381   0.0004   0.6794   0.1960   0.5792    12616
       whit_a_0p20   0.1771   0.1350  -0.0004   0.7188   0.1712   0.6002    12616
       whit_a_0p10   0.1765   0.1342  -0.0008   0.7399   0.1579   0.6079    12616
       whit_a_0p05   0.1770   0.1345  -0.0011   0.7537   0.1492   0.6106    12616
       whit_a_0p01   0.1786   0.1358  -0.0015   0.76

## Randomly drop a two month gap during the growing season (Apr-Sept)

In [15]:
dropped_months = 2
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a two month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a two month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1754   0.1369  -0.0177   0.6398   0.2277   0.6162    12616
        climo_mean   0.1554   0.1171   0.0040   0.6916   0.2142   0.6951    12616
      climo_median   0.1619   0.1124   0.0096   0.7311   0.1928   0.6745    12616
     interp_clim_a   0.1443   0.1140  -0.0068   0.6657   0.2210   0.7460    12616
     interp_clim_b   0.1445   0.1141  -0.0068   0.6654   0.2212   0.7455    12616
     interp_clim_c   0.1444   0.1114  -0.0041   0.6855   0.2103   0.7409    12616
       whit_a_0p50   0.1826   0.1415  -0.0054   0.6560   0.2290   0.5872    12616
       whit_a_0p20   0.1766   0.1358  -0.0028   0.7017   0.2006   0.6173    12616
       whit_a_0p10   0.1743   0.1335  -0.0019   0.7273   0.1839   0.6308    12616
       whit_a_0p05   0.1735   0.1325  -0.0016   0.7447   0.1724   0.6377    12616
       whit_a_0p01   0.1736   0

## Randomly drop a two month gap during the non-growing season (Oct-Mar)

In [16]:
dropped_months = 2
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a two month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a two month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1646   0.1260   0.0340   0.6769   0.2016   0.5384    12582
        climo_mean   0.1497   0.1162   0.0022   0.5698   0.2254   0.5706    12582
      climo_median   0.1541   0.1117  -0.0014   0.5941   0.2091   0.5485    12582
     interp_clim_a   0.1379   0.1070   0.0181   0.6233   0.2135   0.6426    12582
     interp_clim_b   0.1400   0.1083   0.0194   0.6231   0.2149   0.6317    12582
     interp_clim_c   0.1387   0.1059   0.0163   0.6355   0.2054   0.6366    12582
       whit_a_0p50   0.1722   0.1312   0.0171   0.6970   0.1743   0.5079    12582
       whit_a_0p20   0.1724   0.1309   0.0112   0.7207   0.1561   0.5150    12582
       whit_a_0p10   0.1739   0.1320   0.0087   0.7333   0.1470   0.5145    12582
       whit_a_0p05   0.1760   0.1335   0.0072   0.7415   0.1413   0.5110    12582
       whit_a_0p01   0.1797 

## Randomly drop a three month gap during the year

In [17]:
dropped_months = 3
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a three month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a three month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1942   0.1497   0.0035   0.5903   0.2531   0.4975    12616
        climo_mean   0.1504   0.1155  -0.0016   0.6867   0.1893   0.6879    12616
      climo_median   0.1553   0.1103  -0.0007   0.7249   0.1669   0.6718    12616
     interp_clim_a   0.1498   0.1177   0.0009   0.6385   0.2212   0.6950    12616
     interp_clim_b   0.1512   0.1185   0.0018   0.6366   0.2232   0.6887    12616
     interp_clim_c   0.1496   0.1155   0.0014   0.6576   0.2100   0.6934    12616
       whit_a_0p50   0.2052   0.1569   0.0026   0.6462   0.2182   0.4784    12616
       whit_a_0p20   0.2052   0.1563   0.0018   0.6822   0.1955   0.4923    12616
       whit_a_0p10   0.2063   0.1570   0.0014   0.7005   0.1838   0.4968    12616
       whit_a_0p05   0.2078   0.1581   0.0010   0.7124   0.1763   0.4975    12616
       whit_a_0p01   0.2103   0.1602   0.0006   0.

## Randomly drop a three month gap during the growing season (Apr-Sept)

In [18]:
dropped_months = 3
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a three month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a three month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2044   0.1578  -0.0290   0.5737   0.2603   0.4899    12616
        climo_mean   0.1513   0.1156   0.0012   0.7070   0.2000   0.7068    12616
      climo_median   0.1568   0.1107   0.0065   0.7482   0.1774   0.6903    12616
     interp_clim_a   0.1532   0.1217  -0.0139   0.6403   0.2302   0.7084    12616
     interp_clim_b   0.1535   0.1219  -0.0138   0.6398   0.2306   0.7071    12616
     interp_clim_c   0.1523   0.1188  -0.0112   0.6609   0.2188   0.7077    12616
       whit_a_0p50   0.2152   0.1647  -0.0104   0.6073   0.2561   0.4573    12616
       whit_a_0p20   0.2133   0.1625  -0.0054   0.6467   0.2344   0.4774    12616
       whit_a_0p10   0.2128   0.1619  -0.0034   0.6689   0.2213   0.4874    12616
       whit_a_0p05   0.2130   0.1619  -0.0023   0.6840   0.2121   0.4929    12616
       whit_a_0p01   0.2142  

## Randomly drop a three month gap during the non-growing season (Oct-Mar)

In [19]:
dropped_months = 3
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a three month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a three month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1838   0.1410   0.0402   0.6208   0.2413   0.4630    12582
        climo_mean   0.1498   0.1166   0.0003   0.5812   0.2224   0.5880    12582
      climo_median   0.1546   0.1118  -0.0034   0.6062   0.2054   0.5646    12582
     interp_clim_a   0.1464   0.1142   0.0202   0.6010   0.2319   0.6142    12582
     interp_clim_b   0.1489   0.1157   0.0216   0.5991   0.2343   0.6015    12582
     interp_clim_c   0.1469   0.1128   0.0184   0.6135   0.2234   0.6098    12582
       whit_a_0p50   0.1954   0.1489   0.0180   0.6528   0.2022   0.4257    12582
       whit_a_0p20   0.1973   0.1501   0.0109   0.6769   0.1823   0.4296    12582
       whit_a_0p10   0.1998   0.1521   0.0079   0.6898   0.1724   0.4279    12582
       whit_a_0p05   0.2027   0.1544   0.0061   0.6983   0.1661   0.4239    12582
       whit_a_0p01   0.207

## Randomly drop a four month gap during the year

In [20]:
dropped_months = 4
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a four month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a four month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2138   0.1647   0.0065   0.5307   0.2902   0.4064    12616
        climo_mean   0.1517   0.1166   0.0022   0.6794   0.1960   0.6819    12616
      climo_median   0.1563   0.1116   0.0033   0.7160   0.1750   0.6662    12616
     interp_clim_a   0.1591   0.1253   0.0043   0.6050   0.2431   0.6540    12616
     interp_clim_b   0.1609   0.1264   0.0053   0.6021   0.2459   0.6456    12616
     interp_clim_c   0.1586   0.1230   0.0049   0.6233   0.2326   0.6542    12616
       whit_a_0p50   0.2299   0.1752   0.0058   0.5978   0.2490   0.3860    12616
       whit_a_0p20   0.2328   0.1772   0.0047   0.6333   0.2264   0.3948    12616
       whit_a_0p10   0.2353   0.1791   0.0040   0.6513   0.2148   0.3973    12616
       whit_a_0p05   0.2376   0.1810   0.0035   0.6628   0.2073   0.3973    12616
       whit_a_0p01   0.2409   0.1837   0.0028   0.6

## Randomly drop a four month gap during the growing season (Apr-Sept)

In [21]:
dropped_months = 4
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a four month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a four month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2276   0.1762  -0.0373   0.5070   0.2914   0.3913    12616
        climo_mean   0.1509   0.1149   0.0007   0.7118   0.1929   0.7096    12616
      climo_median   0.1563   0.1103   0.0063   0.7525   0.1713   0.6938    12616
     interp_clim_a   0.1631   0.1297  -0.0183   0.6094   0.2422   0.6708    12616
     interp_clim_b   0.1638   0.1302  -0.0179   0.6079   0.2435   0.6672    12616
     interp_clim_c   0.1618   0.1269  -0.0155   0.6297   0.2314   0.6718    12616
       whit_a_0p50   0.2432   0.1854  -0.0153   0.5550   0.2814   0.3577    12616
       whit_a_0p20   0.2465   0.1872  -0.0089   0.5882   0.2656   0.3642    12616
       whit_a_0p10   0.2490   0.1892  -0.0064   0.6056   0.2566   0.3663    12616
       whit_a_0p05   0.2513   0.1910  -0.0051   0.6168   0.2504   0.3664    12616
       whit_a_0p01   0.2545   

## Randomly drop a four month gap during the non-growing season (Oct-Mar)

In [22]:
dropped_months = 4
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a four month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a four month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2019   0.1566   0.0520   0.5883   0.2750   0.4122    12582
        climo_mean   0.1534   0.1183   0.0022   0.5898   0.2244   0.5901    12582
      climo_median   0.1591   0.1139  -0.0011   0.6190   0.2053   0.5646    12582
     interp_clim_a   0.1559   0.1219   0.0271   0.5891   0.2497   0.5896    12582
     interp_clim_b   0.1585   0.1236   0.0288   0.5861   0.2530   0.5773    12582
     interp_clim_c   0.1564   0.1206   0.0255   0.6037   0.2402   0.5861    12582
       whit_a_0p50   0.2178   0.1672   0.0296   0.6375   0.2260   0.3743    12582
       whit_a_0p20   0.2203   0.1690   0.0214   0.6606   0.2053   0.3767    12582
       whit_a_0p10   0.2228   0.1709   0.0177   0.6724   0.1951   0.3757    12582
       whit_a_0p05   0.2254   0.1730   0.0154   0.6800   0.1888   0.3727    12582
       whit_a_0p01   0.2297

## Randomly drop a six month gap during the year

In [23]:
dropped_months = 6
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a six month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a six month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2435   0.1896   0.0061   0.4461   0.3426   0.2841    12616
        climo_mean   0.1523   0.1167  -0.0002   0.6786   0.1950   0.6836    12616
      climo_median   0.1577   0.1122   0.0010   0.7155   0.1739   0.6644    12616
     interp_clim_a   0.1719   0.1357   0.0029   0.5624   0.2688   0.5990    12616
     interp_clim_b   0.1739   0.1372   0.0043   0.5584   0.2726   0.5893    12616
     interp_clim_c   0.1712   0.1335   0.0035   0.5808   0.2582   0.6006    12616
       whit_a_0p50   0.2691   0.2063   0.0048   0.5137   0.3003   0.2599    12616
       whit_a_0p20   0.2781   0.2130   0.0041   0.5421   0.2822   0.2579    12616
       whit_a_0p10   0.2837   0.2174   0.0034   0.5551   0.2737   0.2550    12616
       whit_a_0p05   0.2878   0.2208   0.0028   0.5630   0.2683   0.2524    12616
       whit_a_0p01   0.2928   0.2250   0.0019   0.57

## Randomly drop a six month gap during the growing season (Apr-Sept)

In [24]:
dropped_months = 6
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a six month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a six month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2545   0.1979  -0.0406   0.4132   0.3354   0.2663    12616
        climo_mean   0.1490   0.1131   0.0011   0.7123   0.1855   0.7135    12616
      climo_median   0.1540   0.1086   0.0053   0.7500   0.1655   0.6979    12616
     interp_clim_a   0.1736   0.1380  -0.0197   0.5628   0.2605   0.6213    12616
     interp_clim_b   0.1749   0.1389  -0.0189   0.5600   0.2631   0.6145    12616
     interp_clim_c   0.1723   0.1354  -0.0177   0.5816   0.2504   0.6236    12616
       whit_a_0p50   0.2796   0.2137  -0.0284   0.4648   0.3146   0.2328    12616
       whit_a_0p20   0.2884   0.2198  -0.0229   0.4907   0.3035   0.2297    12616
       whit_a_0p10   0.2940   0.2240  -0.0207   0.5031   0.2977   0.2267    12616
       whit_a_0p05   0.2983   0.2273  -0.0196   0.5105   0.2941   0.2237    12616
       whit_a_0p01   0.3038   0

## Randomly drop a six month gap during the non-growing season (Oct-Mar)

In [25]:
dropped_months = 6
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue

    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a six month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a six month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2307   0.1806   0.0444   0.5145   0.3262   0.3308    12582
        climo_mean   0.1518   0.1168   0.0014   0.6439   0.2081   0.6532    12582
      climo_median   0.1568   0.1123   0.0004   0.6783   0.1872   0.6332    12582
     interp_clim_a   0.1679   0.1321   0.0229   0.5792   0.2671   0.5839    12582
     interp_clim_b   0.1707   0.1340   0.0246   0.5750   0.2713   0.5709    12582
     interp_clim_c   0.1676   0.1301   0.0224   0.5964   0.2567   0.5851    12582
       whit_a_0p50   0.2535   0.1960   0.0324   0.5987   0.2654   0.3120    12582
       whit_a_0p20   0.2617   0.2019   0.0264   0.6314   0.2404   0.3108    12582
       whit_a_0p10   0.2670   0.2060   0.0232   0.6460   0.2288   0.3077    12582
       whit_a_0p05   0.2711   0.2094   0.0210   0.6541   0.2218   0.3039    12582
       whit_a_0p01   0.2765 

### Old plotting function code

In [26]:
# sns.pairplot(
#     output_df[['original'] + comparison_cols], corner=True, kind='reg',
#     plot_kws={'scatter_kws': {'s': 2, 'alpha': 0.2}, 'line_kws': {'color': 'red'}},
# )

In [27]:
# sns.pairplot(output_df[['original'] + comparison_cols], corner=True, kind='hist')

In [28]:
# sns.pairplot(output_df[['original'] + comparison_cols], corner=True, kind='kde')

In [29]:
# g = sns.pairplot(
#     output_df, x_vars = ['original'], y_vars = comparison_cols, kind='reg', 
#     plot_kws={'scatter_kws': {'s': 1, 'alpha': 0.1}, 'line_kws': {'color': 'red'}},
# )
# for ax in g.axes.flatten():
#     if ax:
#         ax.set_xlim(0, 1.2)
#         ax.set_ylim(0, 1.2)
#         ax.set_aspect('equal')
#         ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2])
#         ax.axline((0, 0), slope=1, color='gray', linestyle='--')  # linewidth=1
# plt.show()

In [30]:
# sns.pairplot(
#     output_df[['original'] + comparison_cols], corner=True, kind='reg',
#     plot_kws={'scatter_kws': {'s': 2, 'alpha': 0.1}, 'line_kws': {'color': 'red'}},
# )